In [1]:
import numpy as np
import pandas as pd
import os
os.chdir('./data')
import scipy
import matplotlib.pyplot as plt

In this jupyter notebook file, we show step-by-step process of creating variables to be used. First of all, we obtained the meteorology data from `ronaa` package and selected 5 cities from the four countries; the US, Switzerland, Japan, and Korea. The concatenated data is saved as `longdata.csv`.

In [2]:
df = pd.read_csv('./longdata.csv')
df.head()

,location,year,stationid,lat,long,alt,bloom_date,bloom_doy,date,tmax,tmin,prcp,tavg,month
0,29890,2019,CA001108910,48.91993,-122.640564,10.0,5/11/2019,131,10/1/2018,178.0,125.0,50.0,151.0,-2
1,29890,2019,CA001108910,48.91993,-122.640564,10.0,5/11/2019,131,10/2/2018,153.0,69.0,6.0,111.0,-2
2,29890,2019,CA001108910,48.91993,-122.640564,10.0,5/11/2019,131,10/3/2018,122.0,43.0,0.0,83.0,-2
3,29890,2019,CA001108910,48.91993,-122.640564,10.0,5/11/2019,131,10/4/2018,135.0,21.0,0.0,78.0,-2
4,29890,2019,CA001108910,48.91993,-122.640564,10.0,5/11/2019,131,10/5/2018,151.0,45.0,2.0,98.0,-2


We first define `Date_doy` which is the day of year. For October, November, and December of the previous year, their doy would be subtracted by 365 so that they are number of days from the beginning of the following year.

In [3]:
def define_doy(char):
    doy = pd.Period(char, freq='D').day_of_year
    return(doy)

df['Date_doy'] = df.date.apply(define_doy)

In [4]:
df.Date_doy = [x if x < 250 else x-365 for x in df.Date_doy]

Then we define the four new variables `tavg_moving`, `tmin_moving`, `tmax_moving`, and `tavg_below_5`.
The first three variables are the 14 days moving averages of the temperature data, and tavg_below_5 finds number of accumulate days when its `tavg` is below 5

In [5]:
# 14 days rolling moving average of temperature data and total number of days having tavg below 5
df['tavg_moving'] = df.groupby(['location','year'])['tavg'].transform(lambda x: x.rolling(14, 14).mean()).round(4)
df['tmin_moving'] = df.groupby(['location','year'])['tmin'].transform(lambda x: x.rolling(14, 14).mean()).round(4)
df['tmax_moving'] = df.groupby(['location','year'])['tmax'].transform(lambda x: x.rolling(14, 14).mean()).round(4)
df['tavg_below_5'] = df.groupby(['location','year'])['tavg'].transform(lambda x: x.le(5).sum())

We also define `tavg_above_10`, which is the number of days when `tavg` is above 10 between December and March 10.

In [6]:
df['tavg_above_10'] = df.tavg.ge(10)
new_above_5 = df.loc[df.Date_doy.lt(70) & df.Date_doy.gt(-30),:].groupby(['location','year'])['tavg_above_10'].agg(sum).reset_index()
df = df.drop(['tavg_above_10'], axis = 1)
df = df.merge(new_above_5, on = ['location','year'], how = 'left')

For `tavg_moving`, `tmin_moving`, and `tmax_moving`, our interest is to find the minimum of it and the doy of the last day of the 14 days period. To extract the information, we group by location and year, and found the minimum

In [7]:
minimums = df.groupby(['location','year'])[['location','year','tavg_moving','tmin_moving','tmax_moving']].transform(lambda x: x.min()).drop_duplicates().reset_index(drop = True)
minimums

,location,year,tavg_moving,tmin_moving,tmax_moving
0,29890,2019,-8.5714,-41.9286,24.8571
1,29890,2020,10.5714,-19.9286,41.0
2,29890,2021,-27.9626,-50.7796,-5.5571
3,32789,2019,-24.7483,-71.4286,12.5714
4,32789,2020,17.6321,-15.0,51.2857
...,...,...,...,...,...
754,washingtondc,2017,23.6429,-12.5,58.9286
755,washingtondc,2018,-52.7413,-86.4286,-17.1429
756,washingtondc,2019,-2.5,-50.6429,41.7857
757,washingtondc,2020,30.7857,-6.2143,71.7857


`mindoy_tavg`, `mindoy_tmin`, and `mindoy_tmax` indicate whether the doy of the 14 days period when its average `tavg`, `tmin`, and `tmax` are at minimum respectively. we first set those values at 0 and replace it with 1 when their moving average is equal to the minimum.

In [8]:
df['mindoy_tavg'] = 0
df['mindoy_tmin'] = 0
df['mindoy_tmax'] = 0

In [9]:
for i, l in enumerate(minimums.location.unique()):
    for y in minimums.loc[minimums.location == l,'year']:
        df.loc[df.loc[(df.location == l) & 
               (df.year == y) & 
               (df.tavg_moving == minimums.loc[(minimums.location == l) & 
                                               (minimums.year == y),'tavg_moving'].values[0]),
               'mindoy_tavg'].index[-1],'mindoy_tavg'] = 1
        
        df.loc[df.loc[(df.location == l) & 
               (df.year == y) & 
               (df.tmax_moving == minimums.loc[(minimums.location == l) & 
                                               (minimums.year == y),'tmax_moving'].values[0]),
               'mindoy_tmax'].index[-1],'mindoy_tmax'] = 1
        
        df.loc[df.loc[(df.location == l) & 
               (df.year == y) & 
               (df.tmin_moving == minimums.loc[(minimums.location == l) & 
                                               (minimums.year == y),'tmin_moving'].values[0]),
               'mindoy_tmin'].index[-1],'mindoy_tmin'] = 1

We pause here and copy the dataset.

In [10]:
copy = df.copy()

Now, we create a long-table which gives minimum of `tavg_moving`,`tmin_moving` and `tmax_moving` as well as their DOY.

In [11]:
i = 0
for var1, var2 in zip(['mindoy_tavg','mindoy_tmin','mindoy_tmax'],
                                   ['tavg_moving','tmin_moving','tmax_moving']):
    subset = df.loc[df[var1].eq(1),:]
    subset = subset[['location','year','Date_doy',var2]]
    melt = subset.melt(id_vars=['location','year','Date_doy'], value_vars=[var2])
    if i == 0:
        output = melt
    else:
        output = pd.concat([output, melt])
    i += 1

In [12]:
output

,location,year,Date_doy,variable,value
0,29890,2019,47,tavg_moving,-8.5714
1,29890,2020,19,tavg_moving,10.5714
2,29890,2021,44,tavg_moving,-27.9626
3,32789,2019,33,tavg_moving,-24.7483
4,32789,2020,30,tavg_moving,17.6321
...,...,...,...,...,...
754,washingtondc,2017,10,tmax_moving,58.9286
755,washingtondc,2018,8,tmax_moving,-17.1429
756,washingtondc,2019,23,tmax_moving,41.7857
757,washingtondc,2020,30,tmax_moving,71.7857


By pivoting the table above, we can obtain a table which gives `Date_doy_tavg`, `Date_doy_tmax`, and `Date_doy_tmin`, which are identical to `mindoy_tavg`,`mindoy_tmax`,and `mindoy_tmin` respectively.

In [13]:
tdoy = pd.pivot_table(output, values='Date_doy', index=['location', 'year'],
                    columns=['variable']).reset_index().rename(columns = {'tavg_moving':'Date_doy_tavg',
                                                                         'tmax_moving':'Date_doy_tmax',
                                                                         'tmin_moving':'Date_doy_tmin'})

In [14]:
tdoy

variable,location,year,Date_doy_tavg,Date_doy_tmax,Date_doy_tmin
0,29890,2019,47,47,47
1,29890,2020,19,19,19
2,29890,2021,44,44,44
3,32789,2019,33,22,33
4,32789,2020,30,30,30
...,...,...,...,...,...
754,washingtondc,2017,11,10,11
755,washingtondc,2018,8,8,9
756,washingtondc,2019,34,23,33
757,washingtondc,2020,-6,30,30


The following steps will finally generate a table that each year and location contains a single unique entry.

In [15]:
df = df.merge(tdoy, on = ['location','year'], how = 'left')

In [16]:
# tavg_moving, tmin_moving, and tmax_moving are 0 or NA if they are not at its minimum
df.head()

,location,year,stationid,lat,long,alt,bloom_date,bloom_doy,date,tmax,...,tmin_moving,tmax_moving,tavg_below_5,tavg_above_10,mindoy_tavg,mindoy_tmin,mindoy_tmax,Date_doy_tavg,Date_doy_tmax,Date_doy_tmin
0,29890,2019,CA001108910,48.91993,-122.640564,10.0,5/11/2019,131,10/1/2018,178.0,...,NaN,NaN,10,86.0,0,0,0,47,47,47
1,29890,2019,CA001108910,48.91993,-122.640564,10.0,5/11/2019,131,10/2/2018,153.0,...,NaN,NaN,10,86.0,0,0,0,47,47,47
2,29890,2019,CA001108910,48.91993,-122.640564,10.0,5/11/2019,131,10/3/2018,122.0,...,NaN,NaN,10,86.0,0,0,0,47,47,47
3,29890,2019,CA001108910,48.91993,-122.640564,10.0,5/11/2019,131,10/4/2018,135.0,...,NaN,NaN,10,86.0,0,0,0,47,47,47
4,29890,2019,CA001108910,48.91993,-122.640564,10.0,5/11/2019,131,10/5/2018,151.0,...,NaN,NaN,10,86.0,0,0,0,47,47,47


In [17]:
'''
We make sure that tavg_moving is not 0 if its mindoy_tavg is equal to 1, 
because we will drop the cases where the product of the two variables is 1
'''
df.loc[(df.tavg_moving.eq(0) & df.mindoy_tavg.eq(1)), 'tavg_moving'] = .001

In [18]:
# tavg_moving, tmin_moving, and tmax_moving will be 0 if they are not at its minimum
df['tavg_moving'] = df.tavg_moving*df.mindoy_tavg
df['tmin_moving'] = df.tmin_moving*df.mindoy_tmin
df['tmax_moving'] = df.tmax_moving*df.mindoy_tmax

In [19]:
df = df.loc[(df.tavg_moving.ne(0) | df.tmin_moving.ne(0) | df.tmax_moving.ne(0))&
       df.tavg_moving.notnull() & df.tmin_moving.notnull() & df.tmax_moving.notnull(),:]

In [20]:
df.head()

,location,year,stationid,lat,long,alt,bloom_date,bloom_doy,date,tmax,...,tmin_moving,tmax_moving,tavg_below_5,tavg_above_10,mindoy_tavg,mindoy_tmin,mindoy_tmax,Date_doy_tavg,Date_doy_tmax,Date_doy_tmin
138,29890,2019,CA001108910,48.91993,-122.640564,10.0,5/11/2019,131,2/16/2019,72.0,...,-41.9286,24.8571,10,86.0,1,1,1,47,47,47
322,29890,2020,CA001108910,48.91993,-122.640564,10.0,5/9/2020,130,1/19/2020,73.0,...,-19.9286,41.0000,6,93.0,1,1,1,19,19,19
498,29890,2021,CA001108910,48.91993,-122.640564,10.0,4/30/2021,120,2/13/2021,5.0,...,-50.7796,-5.5571,16,24.0,1,1,1,44,44,44
900,32789,2019,USW00094728,40.73082,-73.997330,6.0,4/16/2019,106,1/22/2019,-5.0,...,-0.0000,12.5714,72,124.0,0,0,1,33,22,33
911,32789,2019,USW00094728,40.73082,-73.997330,6.0,4/16/2019,106,2/2/2019,11.0,...,-71.4286,0.0000,72,124.0,1,1,0,33,22,33


In [21]:
# We don't need the variables in the drop method
df = df.drop(['stationid','date','tmax','tmin','prcp','tavg','month','Date_doy'], axis = 1)

In [22]:
# group by the variables and re-define the following variables: tavg_moving, tmin_moving, and tmax_moving
df = df.groupby(['location','lat','long','alt','year',
            'bloom_date','bloom_doy','Date_doy_tavg',
            'Date_doy_tmax','Date_doy_tmin','tavg_below_5', 'tavg_above_10']).aggregate({'tavg_moving':min,
                                                        'tmin_moving':min,
                                                        'tmax_moving':min}).reset_index()

---

Now we would like to define average of `tmin`, `tmax`, and `tavg` of each month and count of the days when `prcp` is above 0 in a given month. Since the table `df` no longer has daily temperature and precipitation data, we use the `copy` table. 

In [23]:
copy2 = copy.loc[copy.month.le(2),['location','year','month','tavg','tmin','tmax','prcp']]

In [24]:
copy2 = copy2.groupby(['location','year','month']).aggregate({'prcp': lambda x: (x>0).sum(),
                                                     'tmin': np.mean,
                                                     'tmax': np.mean,
                                                     'tavg': np.mean}).reset_index()

In [25]:
copy2.head()

,location,year,month,prcp,tmin,tmax,tavg
0,29890,2019,-2,17.0,63.677419,148.322581,105.967742
1,29890,2019,-1,19.0,52.233333,117.866667,84.966667
2,29890,2019,0,20.0,31.516129,89.129032,60.322581
3,29890,2019,1,15.0,28.290323,95.032258,61.548387
4,29890,2019,2,4.0,-17.964286,48.250000,15.214286


We pivot the table to have one unique entry for a given year and location

The variable names followed by dot and the month value

In [26]:
copy2 = pd.pivot_table(copy2, values=['tmin','tmax','tavg','prcp'], index=['location', 'year'],
                    columns=['month']).reset_index()
colnames = [x+'.'+y if y != '' else x for x,y in zip(copy2.columns.droplevel(1).astype(str), copy2.columns.droplevel(0).astype(str))]
copy2.columns = colnames

In [27]:
copy2.head()

,location,year,prcp.-2,prcp.-1,prcp.0,prcp.1,prcp.2,tavg.-2,tavg.-1,tavg.0,...,tmax.-2,tmax.-1,tmax.0,tmax.1,tmax.2,tmin.-2,tmin.-1,tmin.0,tmin.1,tmin.2
0,29890,2019,17.0,19.0,20.0,15.0,4.0,105.967742,84.966667,60.322581,...,148.322581,117.866667,89.129032,95.032258,48.250000,63.677419,52.233333,31.516129,28.290323,-17.964286
1,29890,2020,14.0,14.0,23.0,22.0,19.0,92.741935,72.933333,58.419355,...,130.000000,108.600000,79.129032,75.612903,87.379310,55.741935,37.200000,37.548387,20.161290,24.793103
2,29890,2021,14.0,9.0,NaN,NaN,10.0,108.483871,64.348175,NaN,...,137.387097,90.794048,NaN,NaN,40.176190,79.516129,38.142460,NaN,NaN,-11.319558
3,32789,2019,22.0,30.0,26.0,20.0,24.0,140.055252,67.824363,44.636134,...,174.935484,98.533333,74.709677,39.129032,59.214286,110.419355,39.233333,15.806452,-32.677419,-11.678571
4,32789,2020,15.0,9.0,14.0,9.0,9.0,151.951782,64.310026,34.899526,...,190.225806,103.400000,63.838710,73.548387,80.551724,119.935484,28.766667,6.677419,6.290323,10.172414


Now, we merge the table with the table `df`. We name the merged table `output`

In [28]:
output= df.merge(copy2.dropna(), how = 'right', on = ['location','year'])

In [29]:
output.head()

,location,lat,long,alt,year,bloom_date,bloom_doy,Date_doy_tavg,Date_doy_tmax,Date_doy_tmin,...,tmax.-2,tmax.-1,tmax.0,tmax.1,tmax.2,tmin.-2,tmin.-1,tmin.0,tmin.1,tmin.2
0,29890,48.91993,-122.640564,10.0,2019,5/11/2019,131,47,47,47,...,148.322581,117.866667,89.129032,95.032258,48.250000,63.677419,52.233333,31.516129,28.290323,-17.964286
1,29890,48.91993,-122.640564,10.0,2020,5/9/2020,130,19,19,19,...,130.000000,108.600000,79.129032,75.612903,87.379310,55.741935,37.200000,37.548387,20.161290,24.793103
2,32789,40.73082,-73.997330,6.0,2019,4/16/2019,106,33,22,33,...,174.935484,98.533333,74.709677,39.129032,59.214286,110.419355,39.233333,15.806452,-32.677419,-11.678571
3,32789,40.73082,-73.997330,6.0,2020,4/27/2020,118,30,30,30,...,190.225806,103.400000,63.838710,73.548387,80.551724,119.935484,28.766667,6.677419,6.290323,10.172414
4,32789,40.73082,-73.997330,6.0,2021,4/14/2021,104,34,52,33,...,177.258065,154.600000,72.709677,42.709677,38.678571,110.322581,78.200000,7.838710,-11.483871,-13.857143


Now we define additional geographical variables.
`long.y` is the sine of longitude and `long.x` is cosine of longitude

In [30]:
output['long.y'] =  output.long.apply(lambda x: np.sin(np.pi/180*x))
output['long.x'] =  output.long.apply(lambda x: np.cos(np.pi/180*x))

In [31]:
copy3 = output[['location','year']].merge(copy)

In [32]:
output.head()

,location,lat,long,alt,year,bloom_date,bloom_doy,Date_doy_tavg,Date_doy_tmax,Date_doy_tmin,...,tmax.0,tmax.1,tmax.2,tmin.-2,tmin.-1,tmin.0,tmin.1,tmin.2,long.y,long.x
0,29890,48.91993,-122.640564,10.0,2019,5/11/2019,131,47,47,47,...,89.129032,95.032258,48.250000,63.677419,52.233333,31.516129,28.290323,-17.964286,-0.842071,-0.539367
1,29890,48.91993,-122.640564,10.0,2020,5/9/2020,130,19,19,19,...,79.129032,75.612903,87.379310,55.741935,37.200000,37.548387,20.161290,24.793103,-0.842071,-0.539367
2,32789,40.73082,-73.997330,6.0,2019,4/16/2019,106,33,22,33,...,74.709677,39.129032,59.214286,110.419355,39.233333,15.806452,-32.677419,-11.678571,-0.961249,0.275682
3,32789,40.73082,-73.997330,6.0,2020,4/27/2020,118,30,30,30,...,63.838710,73.548387,80.551724,119.935484,28.766667,6.677419,6.290323,10.172414,-0.961249,0.275682
4,32789,40.73082,-73.997330,6.0,2021,4/14/2021,104,34,52,33,...,72.709677,42.709677,38.678571,110.322581,78.200000,7.838710,-11.483871,-13.857143,-0.961249,0.275682


Lastly, we define `slope`,`dg2_coef`,`intc`,`tavg.3`,`tmin.3`,and `tmax.3` variables. `slope` and `intc` is found from the linear fit between `Date_doy` and `tavg` between the `Date_doy_tavg` and March 10th. Similarly, `dg2_coef` is found from the quadratic fit between `Date_doy_tavg` and March 10th. If the `Date_doy_tavg` is after 2/28, we just fit linear and quadratic model with days between 1/31 and 3/10. For `tavg.3`,`tmin.3`, and `tmax.3`, we find the average of `tavg`, `tmin` and `tmax` betwen March 1 and March 10.

In [33]:
slope = []
acc = []
intercept = []
march = []
march1 = []
march2 = []
for l,y in zip(output.location, output.year):
    threshold = output.loc[output.location.eq(l) & output.year.eq(y),'Date_doy_tavg'].values[0]
    if threshold > 60:
        ss = copy3.loc[copy3.location.eq(l) & copy3.year.eq(y) & copy3.Date_doy.ge(30) & 
                       copy3.Date_doy.le(70),['Date_doy','tavg','tmin','tmax']]
        acc.append(np.polyfit(x = ss.Date_doy, y = ss.tavg, deg = 2)[0])
        slope.append(np.polyfit(x = ss.Date_doy, y = ss.tavg, deg = 1)[0])
        intercept.append(np.polyfit(x = ss.Date_doy, y = ss.tavg, deg = 1)[1])
        ss = copy3.loc[copy3.Date_doy.gt(59) & 
               copy3.Date_doy.le(70),['Date_doy','tavg','tmin','tmax']]
        march.append(ss.loc[ss.Date_doy.gt(59),'tavg'].mean())
        march1.append(ss.loc[ss.Date_doy.gt(59),'tmax'].mean())
        march2.append(ss.loc[ss.Date_doy.gt(59),'tmin'].mean())
    else:
        ss = copy3.loc[copy3.location.eq(l) & copy3.year.eq(y) & copy3.Date_doy.ge(threshold) & 
                       copy3.Date_doy.le(70),['Date_doy','tavg','tmin','tmax']]
        acc.append(np.polyfit(x = ss.Date_doy, y = ss.tavg, deg = 2)[0])
        slope.append(np.polyfit(x = ss.Date_doy, y = ss.tavg, deg = 1)[0])
        intercept.append(np.polyfit(x = ss.Date_doy, y = ss.tavg, deg = 1)[1])
        march.append(ss.loc[ss.Date_doy.gt(59),'tavg'].mean())
        march1.append(ss.loc[ss.Date_doy.gt(59),'tmax'].mean())
        march2.append(ss.loc[ss.Date_doy.gt(59),'tmin'].mean())

In [34]:
output['slope'] = slope
output['dg2_coef'] = acc
output['intc'] = intercept
output['tavg.3'] = march
output['tmin.3'] = march2
output['tmax.3'] = march1

In [35]:
output.head()

,location,lat,long,alt,year,bloom_date,bloom_doy,Date_doy_tavg,Date_doy_tmax,Date_doy_tmin,...,tmin.1,tmin.2,long.y,long.x,slope,dg2_coef,intc,tavg.3,tmin.3,tmax.3
0,29890,48.91993,-122.640564,10.0,2019,5/11/2019,131,47,47,47,...,28.290323,-17.964286,-0.842071,-0.539367,0.129565,-0.028945,27.253768,35.181818,-10.000000,80.272727
1,29890,48.91993,-122.640564,10.0,2020,5/9/2020,130,19,19,19,...,20.161290,24.793103,-0.842071,-0.539367,-0.250320,0.018789,73.023862,58.727273,27.272727,90.272727
2,32789,40.73082,-73.997330,6.0,2019,4/16/2019,106,33,22,33,...,-32.677419,-11.678571,-0.961249,0.275682,-1.047027,0.064543,75.590678,8.663593,-22.000000,39.545455
3,32789,40.73082,-73.997330,6.0,2020,4/27/2020,118,30,30,30,...,6.290323,10.172414,-0.961249,0.275682,1.164496,0.122699,-4.880522,78.481070,40.454545,121.181818
4,32789,40.73082,-73.997330,6.0,2021,4/14/2021,104,34,52,33,...,-11.483871,-13.857143,-0.961249,0.275682,1.809549,0.112638,-72.127563,39.654925,-2.909091,86.363636
